In [1]:
#Importo le librerie necessarie
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Calcoliamo il numero di pagine web principali che presenta il sito dei competitor
req=requests.get("https://www.lumberjack.com/it/campaign/man?category=shoes")
soup=BeautifulSoup(req.text)
n_pages=int(soup.find_all("a",class_="page-link")[-2].text)
#Andiamo ad acquisire tutti i link alle pagine di dettaglio dei prodotti in ogni pagina web principale, memorizzandoli nella lista links
url_base="https://www.lumberjack.com/it/campaign/man?category=shoes&page="
links=[]
for page in range(1,n_pages+1):
    req=requests.get(url_base+str(page))
    soup=BeautifulSoup(req.text)
    HTML_elements_page=soup.find_all("div",class_="product__image")
    links_page=[]
    for element in HTML_elements_page:
       for child in element.children:
           if child.name!="a":
               continue
           link="https://www.lumberjack.com/"+child.get("href")
           links_page.append(link)
    links.extend(links_page)
    links_page=[]
k=-1                   
for link in links:
    k=k+1
    print(f"{k}  {link}")
        

0  https://www.lumberjack.com//it/product/marvin-sneakers-man-sd0000000000135386?campaign=c_468&ref_screen_code=category 
1  https://www.lumberjack.com//it/product/marvin-sneakers-man-sd0000000000113513?campaign=c_468&ref_screen_code=category 
2  https://www.lumberjack.com//it/product/marvin-sneakers-man-sd0000000000113514?campaign=c_468&ref_screen_code=category 
3  https://www.lumberjack.com//it/product/wilson-sneakers-man-sd0000000000135083?campaign=c_468&ref_screen_code=category 
4  https://www.lumberjack.com//it/product/wilson-sneakers-man-sd0000000000135081?campaign=c_468&ref_screen_code=category 
5  https://www.lumberjack.com//it/product/wilson-sneakers-man-sd0000000000135082?campaign=c_468&ref_screen_code=category 
6  https://www.lumberjack.com//it/product/athens-sneakers-man-sd0000000000135501?campaign=c_468&ref_screen_code=category 
7  https://www.lumberjack.com//it/product/athens-sneakers-man-sd0000000000135502?campaign=c_468&ref_screen_code=category 
8  https://www.lumberjac

In [3]:
#Verifico di aver estratto il numero esatto di siti
len(links)

300

In [43]:
products_info=[]
k=0
for link in links:
    product_info={}
    k=k+1
    req=requests.get(link)
    soup=BeautifulSoup(req.text)
    #if soup.find_all("div",class_="row detail__properties")
    elements=soup.find_all("div",class_="row detail__properties")[0].text.split()
    marchio=elements[0]
    modello=elements[1]
    nome_prodotto=soup.find_all("h1",class_="product__name description text-capital text-capitalize order-1 js-product-name")[0].text.strip()
    prezzo=soup.find_all("span",class_="product__prices-sale")[0].text[:5]
    if soup.find_all("div",class_="detail__description-content w-100")!=[]:
        description=soup.find_all("div",class_="detail__description-content w-100")[0].text.strip()
    else:
        description=""
    product_info["description"]=description
    print(k)
    
    for i in range(len(elements)):
        if elements[i]=="Colore":
            colore=elements[i+1]
        elif elements[i]=="Materiale":
            materiale=elements[i+1]
    """colore=elements.split()[5]
    materiale=elements.split()[10]"""
    product_info={}
    product_info["nome_prodotto"]=nome_prodotto
    product_info["marchio"]=marchio
    product_info["modello"]=modello
    #product_info["descrizione"]=descrizione
    product_info["materiale"]=materiale
    product_info["colore"]=colore
    product_info["prezzo"]=prezzo
    products_info.append(product_info)
    size_elements=soup.find_all("label",class_="detail__sizes-label")
    all_sizes=[]
    for size_element in size_elements:
        size=int(size_element.text.strip())
        all_sizes.append(size)
    size_elements=soup.find_all("label",class_="detail__sizes-label disabled not-stock")
    not_available_sizes=[]
    for size_element in size_elements:
        size=int(size_element.text.strip())
        product_info[size]=0
        not_available_sizes.append(size)
    for size in all_sizes:
        if size not in not_available_sizes:
            product_info[size]=1
df_Lumberjack=pd.DataFrame(products_info)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [32]:
req=requests.get("https://www.lumberjack.com/it/product/marcelo-lace-up-shoes-man-sd0000000000113671?campaign=c_468&")
soup=BeautifulSoup(req.text)
size_elements=soup.find_all("label",class_="detail__sizes-label disabled not-stock")
for size in size_elements:
    print(size.text.strip())


39
44
45
46


In [41]:
req=requests.get("https://www.lumberjack.com/it/product/marcelo-lace-up-shoes-man-sd0000000000113671?campaign=c_468&")
soup=BeautifulSoup(req.text)
size_elements=soup.find_all("label",class_="detail__sizes-label")
for size in size_elements:
    print(type(size.text.strip()))


 

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [44]:
df_Lumberjack

nome_prodotto                   marchio    modello  \
0        MARVIN SNEAKERS UOMO  MarchioLumberjackModello     MARVIN   
1        MARVIN SNEAKERS UOMO  MarchioLumberjackModello     MARVIN   
2        MARVIN SNEAKERS UOMO  MarchioLumberjackModello     MARVIN   
3        WILSON SNEAKERS UOMO  MarchioLumberjackModello     WILSON   
4        WILSON SNEAKERS UOMO  MarchioLumberjackModello     WILSON   
..                        ...                       ...        ...   
295  NAVIGATOR MOCASSINI UOMO  MarchioLumberjackModello  NAVIGATOR   
296       MARVIN SNEAKER UOMO  MarchioLumberjackModello     MARVIN   
297       MARVIN SNEAKER UOMO  MarchioLumberjackModello     MARVIN   
298        RAUL SNEAKERS UOMO  MarchioLumberjackModello       RAUL   
299   GABLE CASUAL SHOES UOMO  MarchioLumberjackModello      GABLE   

           materiale        colore prezzo  39  44  45  46  40  41  42  43  \
0    SUEDE-SYNTHETIC         White  64,99   0   0   0   0   1   1   1   1   
1            LEATHER         White  69,99   0   1   1   1   1   1   1   1   
2            LEATHER          Navy  69,99   0   1   1   0   1   1   1   1   
3        SUEDE-NYLON  Grey/bluette  69,99   0   1   1   0   1   1   1   1   
4        SUEDE-NYLON    White/navy  69,99   0   1   1   0   1   1   1   1   
..               ...           ...    ...  ..  ..  ..  ..  ..  ..  ..  ..   
295            CRAZY     Brown/tan  89,99   0   0   0   0   1   1   0   1   
296    LEATHER-SUEDE          Navy  64,99   0   0   1   0   0   0   1   1   
297    LEATHER-SUEDE         Beige  64,99   0   0   1   0   0   1   1   0   
298  LEATHER-TEXTILE      Universe  89,99   0   0   1   0   0   0   0   0   
299              LWG    Jeans/avio  79,99   0   1   1   0   0   1   0   1   

      47   48  
0    NaN  NaN  
1    NaN  NaN  
2    NaN  NaN  
3    NaN  NaN  
4    NaN  NaN  
..   ...  ...  
295  0.0  0.0  
296  NaN  NaN  
297  NaN  NaN  
298  NaN  NaN  
299  0.0  0.0  

[300 rows x 16 columns]

In [52]:
df_Lumberjack.to_excel("df_LumberjackTaglieCorrette.xlsx")